In [3]:
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [4]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [5]:
train_data = torchvision.datasets.CIFAR10(download= True,root='./data',train = True, transform = transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data\cifar-10-python.tar.gz to ./data


In [7]:
test_data = torchvision.datasets.CIFAR10(root='./data',transform = transform,download = True, train = False)

Files already downloaded and verified


In [8]:
batch_size = 4
train_dataloader = torch.utils.data.DataLoader(dataset= train_data,batch_size = batch_size,shuffle = True)
test_dataloader = torch.utils.data.DataLoader(dataset= test_data,batch_size = batch_size,shuffle=False)

In [17]:
classes = ['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

In [10]:
class ConvNet(nn.Module):
    def __init__(self,num_classes):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,num_classes)
    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        ## Flatten Layer
        x = x.view(-1,16*5*5)
        
        ## Fully Connected Layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
        

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
model = ConvNet(10).to(device)

In [13]:
epochs = 2
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)
loss = nn.CrossEntropyLoss()

In [15]:
num_batches = len(train_dataloader)
for epoch in range(epochs):
    for i ,  (images,labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        
        output = model(images)
        l = loss(output,labels)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        
        if (i%2000)==0:
            print(f"Epoch: {epoch}/{epochs}, Step: {i+1}/{num_batches}, Loss: {l.item():.4f}")

Epoch: 0/2, Step: 1/12500, Loss: 2.3085
Epoch: 0/2, Step: 2001/12500, Loss: 2.2856
Epoch: 0/2, Step: 4001/12500, Loss: 2.2895
Epoch: 0/2, Step: 6001/12500, Loss: 2.3092
Epoch: 0/2, Step: 8001/12500, Loss: 2.3022
Epoch: 0/2, Step: 10001/12500, Loss: 2.2748
Epoch: 0/2, Step: 12001/12500, Loss: 2.0544
Epoch: 1/2, Step: 1/12500, Loss: 2.1143
Epoch: 1/2, Step: 2001/12500, Loss: 1.9370
Epoch: 1/2, Step: 4001/12500, Loss: 1.8557
Epoch: 1/2, Step: 6001/12500, Loss: 1.7780
Epoch: 1/2, Step: 8001/12500, Loss: 1.9883
Epoch: 1/2, Step: 10001/12500, Loss: 1.6016
Epoch: 1/2, Step: 12001/12500, Loss: 1.2876


In [18]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs,1)
        n_samples += labels.size(0)
        n_correct += (predicted==labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label]+=1
            n_class_samples[label]+=1
            
acc = 100.0*(n_correct/n_samples)
print(f"Testing_accuracy: {acc:.4f}")

for i in range(10):
    acc = 100.0*(n_class_correct[i]/n_class_samples[i])
    print(f"Accuracy of {classes[i]}: {acc:.2f}%")
            

Testing_accuracy: 34.2400
Accuracy of plane: 60.80%
Accuracy of car: 35.40%
Accuracy of bird: 3.70%
Accuracy of cat: 10.10%
Accuracy of deer: 13.60%
Accuracy of dog: 45.70%
Accuracy of frog: 65.00%
Accuracy of horse: 47.80%
Accuracy of ship: 18.00%
Accuracy of truck: 42.30%
